In [17]:
#combining all datasets using map function

import glob
import pandas as pd 

#map renewables

df_renew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_renewable/*.csv')))
df_renew.rename(columns={'Country or Area':'Geo'}, inplace=True)

#map nonrenewables

df_nonrenew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_nonrenewable/*.csv')))
df_nonrenew.rename(columns={'Country or Area':'Geo'}, inplace=True)

df = pd.concat([df_renew, df_nonrenew])

#split commodity and transaction into two columns
#defining column to split and which delimiter
ct = df['Commodity - Transaction'].str.split('-')

#assigning new columns
df['Commodity'] = ct.str.get(0)
df['Transaction'] = ct.str.get(1)

#lowercase and remove trailing and leading spaces
df['Commodity'] = df['Commodity'].str.lower()
df['Commodity'] = df['Commodity'].str.strip()

df['Transaction'] = df['Transaction'].str.lower()
df['Transaction'] = df['Transaction'].str.strip()

df = df[df['Transaction'] == 'final consumption']

df.sample(5)

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction
17533,United Kingdom,Vegetal waste - Final consumption,1997.0,Terajoules,7874.000,1.0,vegetal waste,final consumption
14860,Madagascar,Lubricants - Final consumption,2007.0,"Metric tons, thousand",4.469,1.0,lubricants,final consumption
16243,Jordan,Charcoal - Final consumption,2007.0,"Metric tons, thousand",19.000,NaN,charcoal,final consumption
59394,Togo,Motor Gasoline - Final consumption,2008.0,"Metric tons, thousand",177.000,NaN,motor gasoline,final consumption
6620,Denmark,Bitumen - Final consumption,2010.0,"Metric tons, thousand",213.000,NaN,bitumen,final consumption


In [18]:
import pandas as pd 
unit_conv = pd.read_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_energy_units_conv_2.csv')

df_merge = pd.merge(df,  
                     unit_conv,  
                     on ='Commodity', 
                     how ='left') 

df_merge['TJ'] = df_merge['Quantity'] * df_merge['Conv_mt_to_tj']
df_merge.sample(2)

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
31008,Belgium,Coal Tar - Final consumption,2017.0,"Metric tons, thousand",283.0,NaN,coal tar,final consumption,Coal tar is used primarily for the production ...,"Metric tons, thousand",28.0,nonrenewable,7924.00
50399,Mayotte,Liquefied petroleum gas (LPG) - Final consumption,2005.0,"Metric tons, thousand",1.4,1.0,liquefied petroleum gas (lpg),final consumption,NaN,"Metric tons, thousand",47.3,nonrenewable,66.22


In [19]:

lt_2018 = df_merge[df_merge['Year'] == 2013]
tot_grp_country = lt_2018.groupby(['Geo'])
df_tot_country = tot_grp_country['TJ'].agg(['sum'])

#sum all renewables
lt_2018_r = df_merge[(df_merge['Year'] == 2013) & (df_merge['r_type'] == 'renewable')]
tot_r_grp_country = lt_2018_r.groupby(['Geo'])
df_tot_country_r = tot_r_grp_country['TJ'].agg(['sum'])

#sum all nonrenewables
lt_2018_n = df_merge[(df_merge['Year'] == 2013) & (df_merge['r_type'] == 'nonrenewable')]
tot_n_grp_country = lt_2018_n.groupby(['Geo'])
df_tot_country_n = tot_n_grp_country['TJ'].agg(['sum'])

In [20]:
#merge total, renewable, and nonrenewable dataframes

countries_r = pd.merge(df_tot_country,  
                     df_tot_country_r,  
                     on ='Geo', 
                     how ='left') 

countries_r_n = pd.merge(countries_r,
                    df_tot_country_n,
                    on = 'Geo', 
                    how = 'left')

countries_r_n.rename(columns={'sum_x':'total', 'sum_y': 'renew', 'sum' : 'nonrenew'}, inplace=True)
countries_r_n['percent_renewable'] = (countries_r_n['renew'] / (countries_r_n['renew'] + countries_r_n['nonrenew'])) * 100
countries_r_n.sort_values(by='percent_renewable', ascending=False, inplace=True)
#allsources_r_n['diff'] = allsources_r_n['renew'] + allsources_r_n['nonrenew']#

countries_r_n

,total,renew,nonrenew,percent_renewable
Geo,,,,
Somalia,102277.080960,97936.690960,4340.390000,95.756244
Uganda,468227.770292,445342.173805,22885.596487,95.112294
Dem. Rep. of the Congo,759556.515868,711877.515868,47679.000000,93.722784
United Rep. of Tanzania,617675.549532,570756.549532,46919.000000,92.403941
Zambia,278842.131252,257466.731252,21375.400000,92.334229
...,...,...,...,...
Nauru,355.573000,NaN,355.573000,NaN
Palau,2083.020000,NaN,2083.020000,NaN
Turks and Caicos Islands,6.445320,6.445320,NaN,NaN


In [21]:
#export results to csv for tableau viz, keep commented out

#countries_r_n.to_csv('countries_percent_renewable_2013.csv')

In [22]:


from matplotlib import pyplot as plt 

for geo in df_merge['Geo'].unique() :
    df_r_type_country = df_merge[df_merge['Geo'] == geo]
    r_grp_country = df_r_type_country.groupby(['r_type'])
    sum_r_type = r_grp_country['TJ'].agg(['sum'])
    sum_r_type['percent'] = (sum_r_type['sum'] / sum_r_type['sum'].sum()) * 100
    sum_r_type.head() 
    
    
    
    pie = sum_r_type['sum']
    labels = ['nonrenewable', 'renewable']

    #plt.style.use("fivethirtyeight")

    plt.pie(pie, labels=labels, wedgeprops={'edgecolor':'black'})
    plt.title(str(geo))
    #plt.tight_layout()

    #plt.savefig('worldwide_allsources_2018_pie')
    #plt.show()

ValueError: 'label' must be of length 'x'